<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/q%26a/langchain_cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cohere langchain==0.0.91 chromadb==0.3.2 tfds-nightly> /dev/null

In [2]:
import os
import random

from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain

In [11]:
os.environ["COHERE_API_KEY"] = ""

In [ ]:
# Usaremos el discurso de graduación de la Universidad de Stanford de Steve Jobs como ejemplo.
# Enlace: https://news.stanford.edu/2005/06/12/youve-got-find-love-jobs-says/

!wget 'https://docs.google.com/uc?export=download&id=1f1INWOfJrHTFmbyF_0be5b4u_moz3a4F' -O steve-jobs-commencement.txt

In [14]:
with open('steve-jobs-commencement.txt') as f:
    text = f.read()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = text_splitter.split_text(text)

In [16]:
embeddings = CohereEmbeddings(model = "multilingual-22-12")
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": f"{i}"} for i in range(len(texts))])

Running Chroma using direct local API.


DEBUG:Chroma:Logger created


Using DuckDB in-memory for database. Data will be transient.


DEBUG:Chroma:Index not found
DEBUG:Chroma:Index saved to {self._save_folder}/index.bin
DEBUG:Chroma:Index saved to {self._save_folder}/index.bin


In [17]:
questions = [
           "What did the author liken The Whole Earth Catalog to?",
           "What was Reed College great at?",
           "What was the author diagnosed with?",
           "What is the key lesson from this article?",
           "What did the article say about Michael Jackson?",
           ]

In [18]:
prompt_template = """Text: {context}

Question: {question}

Answer the question based on the text provided. If the text doesn't contain the answer, reply that the answer is not available."""

In [19]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [20]:
# Generar la respuesta dado el contexto
chain = load_qa_chain(Cohere(model="command-xlarge-nightly", temperature=0), chain_type="stuff", prompt=PROMPT)

for question in questions:
    docs = docsearch.similarity_search(question)
    answer = chain.run(input_documents=docs, question=question)
    answer = answer.replace("\n","").replace("Answer:","")
    print("-"*20,"\n")
    print(f"Question: {question}")
    print(f"Answer: {answer}")

DEBUG:Chroma:time to pre process our knn query: 3.5762786865234375e-06
DEBUG:Chroma:time to run knn query: 0.00022602081298828125
DEBUG:Chroma:time to pre process our knn query: 4.291534423828125e-06
DEBUG:Chroma:time to run knn query: 0.0001995563507080078


-------------------- 

Question: What did the author liken The Whole Earth Catalog to?
Answer: Google
-------------------- 

Question: What was Reed College great at?
Answer: Question: What was Reed College great at? Reed College at that time offered perhaps the best calligraphy instruction in the country.


DEBUG:Chroma:time to pre process our knn query: 4.76837158203125e-06
DEBUG:Chroma:time to run knn query: 0.0003330707550048828
DEBUG:Chroma:time to pre process our knn query: 4.0531158447265625e-06
DEBUG:Chroma:time to run knn query: 0.00011706352233886719


-------------------- 

Question: What was the author diagnosed with?
Answer: Cancer


DEBUG:Chroma:time to pre process our knn query: 5.4836273193359375e-06
DEBUG:Chroma:time to run knn query: 0.0022115707397460938


-------------------- 

Question: What is the key lesson from this article?
Answer: Stay Hungry. Stay Foolish.
-------------------- 

Question: What did the article say about Michael Jackson?
Answer: Stay Hungry. Stay Foolish.Michael Jackson died.


**Referencias**

* https://cohere.ai/
* https://python.langchain.com/en/latest/getting_started/getting_started.html

**Por hacer**

* Añadir descripción del modelo
* Añadir comentarios
* Adaptar a ejemplo en español